In [1]:
import pandas as pd
df = pd.read_csv('fakenews_train.csv')

In [2]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
df = df.dropna()

In [8]:
X = df.drop(columns=['label'])
X.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [9]:
y = df['label']
y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [10]:
X.shape,y.shape

((18285, 4), (18285,))

In [11]:
import tensorflow as tf

In [13]:
tf.__version__

'2.3.1'

In [14]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Dense, LSTM

In [16]:
messages = X.copy()
messages.reset_index(inplace = True)

In [17]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [21]:
ps = PorterStemmer()
corpus = []
for i in range(len(messages)):
    review = re.sub('[^a-zA-z]',' ',messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word)for word in review if word not  in stopwords.words("english")]
    review = ' '.join(review)
    corpus.append(review)

In [19]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic [photos]',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hi

In [22]:
vocb_size = 10000

In [23]:
#onehot_representation
one_hot_ = [one_hot(words,vocb_size)for words in corpus]
print(one_hot_)

[[8362, 1581, 483, 9506, 5632, 7103, 2312, 2799, 108, 4774], [4818, 1201, 1655, 4990, 7789, 9966, 7184], [9369, 4319, 1110, 1812], [1524, 2963, 6118, 891, 789, 2454], [2621, 7789, 1193, 3591, 5020, 9983, 7789, 3187, 3354, 7305], [7799, 2212, 6097, 4398, 4042, 7809, 5317, 7591, 1750, 6748, 5027, 2067, 8877, 7164, 7184], [598, 9941, 3493, 3095, 523, 7861, 6263, 5731, 2719, 6170, 5673], [836, 1958, 9264, 8812, 2716, 321, 7809, 7648, 2719, 6170, 5673], [4070, 4324, 9934, 1440, 1840, 9, 6337, 1607, 7809, 8422], [2649, 792, 3146, 5362, 7339, 2010, 4785, 2841], [1756, 7324, 6264, 6215, 9742, 6733, 3564, 5592, 7454, 9750, 9788], [891, 668, 5632, 9, 7809, 2716], [791, 5107, 3693, 2245, 944, 9010, 7644, 4560, 3727], [5077, 4070, 8854, 4712, 1026, 6093, 450, 2719, 6170, 5673], [6618, 5185, 487, 3493, 4850, 2719, 6170, 5673], [1645, 8725, 7707, 2206, 5254, 2671, 6688, 7590, 4662, 3613], [2129, 7886, 1201], [8274, 916, 6626, 2449, 7809, 4071, 1218, 7184], [5807, 171, 1655, 1529, 4650, 3149, 5354, 6

In [27]:
#embedding_documentation
sent_length=20
embedded_docs=pad_sequences(one_hot_,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2799  108 4774]
 [   0    0    0 ... 7789 9966 7184]
 [   0    0    0 ... 4319 1110 1812]
 ...
 [   0    0    0 ... 2719 6170 5673]
 [   0    0    0 ...   36 7927 2363]
 [   0    0    0 ... 4489 6291 1046]]


In [28]:
#creating_model 
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(vocb_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            400000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 456,501
Trainable params: 456,501
Non-trainable params: 0
_________________________________________________________________
None


In [29]:
len(embedded_docs),y.shape


(18285, (18285,))

In [30]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)
X_final.shape,y_final.shape



((18285, 20), (18285,))

In [31]:
#train_test_split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [32]:
## Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 13s 67ms/step - loss: 0.3306 - accuracy: 0.8422 - val_loss: 0.1859 - val_accuracy: 0.9226
Epoch 2/10
192/192 [==============================] - 13s 66ms/step - loss: 0.1207 - accuracy: 0.9524 - val_loss: 0.1960 - val_accuracy: 0.9239
Epoch 3/10
192/192 [==============================] - 15s 76ms/step - loss: 0.0728 - accuracy: 0.9749 - val_loss: 0.2393 - val_accuracy: 0.9130
Epoch 4/10
192/192 [==============================] - 12s 63ms/step - loss: 0.0461 - accuracy: 0.9847 - val_loss: 0.2956 - val_accuracy: 0.9181
Epoch 5/10
192/192 [==============================] - 11s 58ms/step - loss: 0.0295 - accuracy: 0.9913 - val_loss: 0.3900 - val_accuracy: 0.9178
Epoch 6/10
192/192 [==============================] - 10s 54ms/step - loss: 0.0168 - accuracy: 0.9949 - val_loss: 0.4445 - val_accuracy: 0.9137
Epoch 7/10
192/192 [==============================] - 10s 54ms/step - loss: 0.0159 - accuracy: 0.9954 - val_loss: 0.4434 - val_accuracy:

In [48]:
#Adding Dropout
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(vocb_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 20, 40)            400000    
_________________________________________________________________
dropout_6 (Dropout)          (None, 20, 40)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dropout_7 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 101       
Total params: 456,501
Trainable params: 456,501
Non-trainable params: 0
_________________________________________________________________


In [49]:
#Performance Metrics And Accuracy
y_pred=model.predict_classes(X_test)


In [50]:
#confusion_matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)



array([[2540,  879],
       [1498, 1118]])

In [51]:
#accuracy_model
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6061309030654516